# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,enewetak,11.3474,162.3373,27.06,80,47,10.49,MH,1682934213
1,1,kapuskasing,49.4169,-82.4331,6.05,100,100,5.66,CA,1682934061
2,2,wailua homesteads,22.0669,-159.3780,23.44,85,40,5.66,US,1682934214
3,3,bethel,41.3712,-73.4140,11.20,87,40,2.57,US,1682934215
4,4,adamstown,-25.0660,-130.1015,25.08,83,77,5.33,PN,1682934215


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] <27), :]
city_wind_df = city_temp_df.loc[city_temp_df["Wind Speed"] < 4.5, :]
city_cloud_df = city_wind_df.loc[city_wind_df["Cloudiness"] == 0, :]

# Drop any rows with null values
city_weather_df = city_cloud_df.dropna(how='any')

# Display sample data
city_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,port isabel,26.0734,-97.2086,21.14,93,0,3.60,US,1682934234
114,114,jumilla,38.4792,-1.3250,22.20,49,0,1.97,ES,1682933846
120,120,al jawf,29.5000,38.7500,23.43,20,0,1.09,SA,1682934286
209,209,sao vicente,-23.9631,-46.3919,24.10,26,0,1.47,BR,1682934341
256,256,bertioga,-23.8544,-46.1386,23.90,85,0,1.78,BR,1682934373
376,376,tazacorte,28.6290,-17.9293,24.54,38,0,3.60,ES,1682934466
470,470,arar,30.9753,41.0381,25.01,18,0,1.54,SA,1682934525
502,502,pontes e lacerda,-15.2261,-59.3353,21.03,95,0,0.55,BR,1682934261
516,516,pimentel,-6.8367,-79.9342,25.16,78,0,1.03,PE,1682934553


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,port isabel,US,26.0734,-97.2086,93,
114,jumilla,ES,38.4792,-1.3250,49,
120,al jawf,SA,29.5000,38.7500,20,
209,sao vicente,BR,-23.9631,-46.3919,26,
256,bertioga,BR,-23.8544,-46.1386,85,
376,tazacorte,ES,28.6290,-17.9293,38,
470,arar,SA,30.9753,41.0381,18,
502,pontes e lacerda,BR,-15.2261,-59.3353,95,
516,pimentel,PE,-6.8367,-79.9342,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel" ,
    "api_key": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    #Save city name
    city = hotel_df.loc[index, "City"]

    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{city} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port isabel - nearest hotel: Queen Isabel Inn
jumilla - nearest hotel: Pio XII
al jawf - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
bertioga - nearest hotel: Pousada Vila da Praia
tazacorte - nearest hotel: App Leyma
arar - nearest hotel: دار أوتيل Dar Otel
pontes e lacerda - nearest hotel: Hotel Avenida
pimentel - nearest hotel: Hospedaje Calle del Sol


,City,Country,Lat,Lng,Humidity,Hotel Name
34,port isabel,US,26.0734,-97.2086,93,Queen Isabel Inn
114,jumilla,ES,38.4792,-1.3250,49,Pio XII
120,al jawf,SA,29.5000,38.7500,20,No hotel found
209,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
256,bertioga,BR,-23.8544,-46.1386,85,Pousada Vila da Praia
376,tazacorte,ES,28.6290,-17.9293,38,App Leyma
470,arar,SA,30.9753,41.0381,18,دار أوتيل Dar Otel
502,pontes e lacerda,BR,-15.2261,-59.3353,95,Hotel Avenida
516,pimentel,PE,-6.8367,-79.9342,78,Hospedaje Calle del Sol


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.2,
    color = "City",
    alpha = 0.5,
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)